# Q3: How Robust Credit Risk Models are Over Time

This script compares [INSERT MODEL NAMES] in their performance in credit risk prediction when trained and tested data differ in economic periods.

### Imports

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import subprocess
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))

from data_processings.datasets import LendingClubDataset
from data_processings.feature_engineering import process_q3_features

### Pre-Processing of Data

Loading Dataset for Accepted Loans

In [2]:
num_samples = 5000
dataloader = LendingClubDataset()
accepted_df = dataloader.load(num_samples)

Feature Construction

In [3]:
new_df = process_q3_features(accepted_df)
new_df

,loan_status,revol_util,purpose,home_ownership,emp_length,term,dti_new,loan_income_ratio,fico_mean
0,Fully Paid,29.7,debt_consolidation,MORTGAGE,10+ years,36 months,5.91,0.065455,677.0
1,Fully Paid,19.2,small_business,MORTGAGE,10+ years,36 months,16.06,0.380000,717.0
2,Fully Paid,56.2,home_improvement,MORTGAGE,10+ years,60 months,13.85,0.317460,697.0
4,Fully Paid,64.5,major_purchase,MORTGAGE,3 years,60 months,25.37,0.099585,697.0
5,Fully Paid,68.4,debt_consolidation,RENT,4 years,36 months,10.20,0.351471,692.0
...,...,...,...,...,...,...,...,...,...
4993,Fully Paid,57.0,other,MORTGAGE,< 1 year,60 months,14.41,0.360000,687.0
4996,Charged Off,67.7,debt_consolidation,MORTGAGE,3 years,60 months,17.63,0.254118,667.0
4997,Fully Paid,98.1,credit_card,RENT,2 years,36 months,35.01,0.073846,687.0
4998,Charged Off,62.1,credit_card,RENT,4 years,60 months,23.13,0.439865,707.0


Feature 

In [ ]:
print(list(accepted_df.columns))

In [ ]:
accepted_df["loan_status"].unique()

In [ ]:
accepted_df[["id"]]

In [ ]:
accepted_df['dti_final'] = accepted_df['dti_joint'].fillna(accepted_df['dti'])

In [ ]:
accepted_df["annual_inc"].unique()
